# Usando datos de ánime para un análisis de sistema de recomendación
  
El dataset consta de el id del ánime, del usuario y de la puntuación que el usuario le dio al ánime

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("anime").getOrCreate()

In [4]:
data = spark.read.csv("C:/Users/solk1/OneDrive/Desktop/Ciencia de datos/pruebas_ML/datos/animes/rating.csv",
                     inferSchema = True, header = True)
data.show()

+-------+--------+------+
|user_id|anime_id|rating|
+-------+--------+------+
|      1|      20|    -1|
|      1|      24|    -1|
|      1|      79|    -1|
|      1|     226|    -1|
|      1|     241|    -1|
|      1|     355|    -1|
|      1|     356|    -1|
|      1|     442|    -1|
|      1|     487|    -1|
|      1|     846|    -1|
|      1|     936|    -1|
|      1|    1546|    -1|
|      1|    1692|    -1|
|      1|    1836|    -1|
|      1|    2001|    -1|
|      1|    2025|    -1|
|      1|    2144|    -1|
|      1|    2787|    -1|
|      1|    2993|    -1|
|      1|    3455|    -1|
+-------+--------+------+
only showing top 20 rows



In [5]:
data.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- anime_id: integer (nullable = true)
 |-- rating: integer (nullable = true)



In [6]:
data.describe().show()

+-------+------------------+-----------------+------------------+
|summary|           user_id|         anime_id|            rating|
+-------+------------------+-----------------+------------------+
|  count|           7813737|          7813737|           7813737|
|   mean|36727.956744640884|8909.072104295294| 6.144029546937656|
| stddev|20997.946118974716|8883.949635880988|3.7278004201098796|
|    min|                 1|                1|                -1|
|    max|             73516|            34519|                10|
+-------+------------------+-----------------+------------------+



In [12]:
train, test = data.randomSplit([0.7, 0.3])

In [14]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [15]:
als = ALS(maxIter = 5, regParam = 0.01, userCol = "user_id", itemCol = "anime_id", ratingCol = "rating")

In [16]:
modelo = als.fit(train)

In [17]:
resultados = modelo.transform(test)
resultados.show()

+-------+--------+------+-----------+
|user_id|anime_id|rating| prediction|
+-------+--------+------+-----------+
|      1|    6773|    -1|  -0.789167|
|      1|    9675|    -1| -1.2019289|
|      1|     355|    -1| -0.3217999|
|      1|    4898|    -1| -1.0431083|
|      1|     442|    -1| -1.0794796|
|      1|    9515|    -1|-0.46411788|
|      1|    1692|    -1| -1.0797834|
|      1|    5667|    -1| -1.0565588|
|      1|    9790|    -1|-0.32600793|
|      1|    6747|    -1| -0.6710637|
|      1|    9330|    -1|-0.24544892|
|      1|    6707|    -1|  -0.670087|
|      1|    4224|    -1|-0.18395989|
|      1|    5781|    -1| -1.0359017|
|      1|    2787|    -1|-0.49720526|
|      1|   10067|    -1| -1.1209084|
|      1|    5958|    -1|-0.61213773|
|      1|    8525|    -1|-0.45144045|
|      1|    8630|    -1|-0.60644346|
|      1|    9136|    -1|-0.70531315|
+-------+--------+------+-----------+
only showing top 20 rows



In [20]:
my_eval = RegressionEvaluator(metricName = "rmse", labelCol = "rating", predictionCol = "prediction")

In [22]:
my_eval.evaluate(resultados)

nan

In [50]:
un_usuario = test.filter("user_id == 1").select("user_id", "anime_id")
un_usuario.show()

+-------+--------+
|user_id|anime_id|
+-------+--------+
|      1|     355|
|      1|     442|
|      1|    1692|
|      1|    2787|
|      1|    4224|
|      1|    4898|
|      1|    5667|
|      1|    5781|
|      1|    5958|
|      1|    6707|
|      1|    6747|
|      1|    6773|
|      1|    8525|
|      1|    8630|
|      1|    9062|
|      1|    9136|
|      1|    9330|
|      1|    9515|
|      1|    9675|
|      1|    9790|
+-------+--------+
only showing top 20 rows



In [53]:
resul_un_usuario = modelo.transform(un_usuario)
resul_un_usuario.show()

+-------+--------+-----------+
|user_id|anime_id| prediction|
+-------+--------+-----------+
|      1|     355| -0.3217999|
|      1|     442| -1.0794796|
|      1|    1692| -1.0797834|
|      1|    2787|-0.49720526|
|      1|    4224|-0.18395989|
|      1|    4898| -1.0431083|
|      1|    5667| -1.0565588|
|      1|    5781| -1.0359017|
|      1|    5958|-0.61213773|
|      1|    6707|  -0.670087|
|      1|    6747| -0.6710637|
|      1|    6773|  -0.789167|
|      1|    8525|-0.45144045|
|      1|    8630|-0.60644346|
|      1|    9062| -1.4828746|
|      1|    9136|-0.70531315|
|      1|    9330|-0.24544892|
|      1|    9515|-0.46411788|
|      1|    9675| -1.2019289|
|      1|    9790|-0.32600793|
+-------+--------+-----------+
only showing top 20 rows



In [26]:
import pandas as pd
from siuba import *
import numpy as np

In [25]:
df = resultados.toPandas()

In [35]:
df >> arrange(_.user_id)

,user_id,anime_id,rating,prediction
1146,1,6773,-1,-0.789167
6278,1,9675,-1,-1.201929
9464,1,20785,-1,-0.723874
27368,1,11757,10,0.191743
42413,1,355,-1,-0.321800
...,...,...,...,...
2337703,73515,1535,10,8.090982
2338889,73515,12413,9,8.840080
2340229,73515,2002,9,10.275136
2343729,73515,3782,10,10.160192


In [34]:
df.isnull().sum()

user_id          0
anime_id         0
rating           0
prediction    1699
dtype: int64

In [37]:
df.dropna(inplace = True)

In [40]:
df["prediction"] = df["prediction"].astype("int64")

In [41]:
df >> arrange(_.user_id)

,user_id,anime_id,rating,prediction
1146,1,6773,-1,0
6278,1,9675,-1,-1
9464,1,20785,-1,0
27368,1,11757,10,0
42413,1,355,-1,0
...,...,...,...,...
2337703,73515,1535,10,8
2338889,73515,12413,9,8
2340229,73515,2002,9,10
2343729,73515,3782,10,10


In [45]:
from sklearn.metrics import mean_squared_error

In [46]:
np.sqrt(mean_squared_error(df["rating"], df["prediction"]))

2.203615539955535

In [54]:
df_un_usuario = resul_un_usuario.toPandas()

In [57]:
df_un_usuario

,user_id,anime_id,prediction
0,1,6773,-0.789167
1,1,9675,-1.201929
2,1,20785,-0.723874
3,1,11757,0.191743
4,1,355,-0.321800
5,1,23321,-0.959734
6,1,29095,-1.157776
7,1,4898,-1.043108
8,1,11759,-0.310607
9,1,442,-1.079480


In [62]:
df_pandas_un_usuario = df[df["user_id"] == 1]
df_pandas_un_usuario.index = df_un_usuario.index
df_pandas_un_usuario

,user_id,anime_id,rating,prediction
0,1,6773,-1,0
1,1,9675,-1,-1
2,1,20785,-1,0
3,1,11757,10,0
4,1,355,-1,0
5,1,23321,-1,0
6,1,29095,-1,-1
7,1,4898,-1,-1
8,1,11759,-1,0
9,1,442,-1,-1


In [63]:
pd.concat([df_pandas_un_usuario, df_un_usuario], axis = 1)

,user_id,anime_id,rating,prediction,user_id,anime_id,prediction
0,1,6773,-1,0,1,6773,-0.789167
1,1,9675,-1,-1,1,9675,-1.201929
2,1,20785,-1,0,1,20785,-0.723874
3,1,11757,10,0,1,11757,0.191743
4,1,355,-1,0,1,355,-0.321800
5,1,23321,-1,0,1,23321,-0.959734
6,1,29095,-1,-1,1,29095,-1.157776
7,1,4898,-1,-1,1,4898,-1.043108
8,1,11759,-1,0,1,11759,-0.310607
9,1,442,-1,-1,1,442,-1.079480


In [68]:
#Viendo a otro usuario:
df[df["user_id"] == 5000]

,user_id,anime_id,rating,prediction
203,5000,8086,9,7
1178,5000,12367,6,5
2541,5000,1903,7,6
9515,5000,4725,8,8
11767,5000,17074,8,9
...,...,...,...,...
213030,5000,26243,7,7
214216,5000,10719,7,7
214217,5000,23199,8,8
216190,5000,4618,8,7
